In [ ]:
!pip install mlflow
import keras
import cv2
import numpy as np
import argparse
import os
import pandas as pd
import scipy.io
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
import mlflow
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.applications import VGG16,InceptionV3
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import AveragePooling2D, Dropout,Flatten, Dense, Input, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from imblearn.metrics import sensitivity_score, specificity_score
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix#,plot_confusion_matrix
import seaborn as sn
import itertools
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import BatchNormalization, Conv2D, MaxPooling2D, UpSampling2D, Add, Activation, Multiply, concatenate
from PIL import Image
from tensorflow.keras.applications import DenseNet169

# Set the MLflow experiment
mlflow.set_experiment("Covid19_Classification using VGG16")

# Start MLflow run
with mlflow.start_run(run_name='Model Mixed1'):

    # Add MLflow tags to track important information
    mlflow.set_tag("model", "VGG16")
    mlflow.set_tag("framework", "Keras")

    height = width = 256

    train_datagen = ImageDataGenerator(rescale=1./255)
    validation_datagen = ImageDataGenerator(rescale=1./255)
    test_datagen = ImageDataGenerator(rescale=1./255)

    epochs = 4
    learning_rate = 1e-4
    BS = 16
    num_classes = 2

    y_true = []
    y_pred = []
    cvscores = []
    pred_prob = []

    train_df = pd.read_csv('../input/hold-out-split-srim/train.csv')
    dev_df = pd.read_csv('../input/hold-out-split-srim/dev.csv')
    test_df = pd.read_csv('../input/hold-out-split-srim/test.csv')
    unseentest_df = pd.read_csv('../input/hold-out-split-srim/test_unseen.csv')

    train_df['class'].replace(to_replace=['PNEUMO', 'NORMAL'], value='OTHER', inplace=True)
    dev_df['class'].replace(to_replace=['PNEUMO', 'NORMAL'], value='OTHER', inplace=True)
    test_df['class'].replace(to_replace=['PNEUMO', 'NORMAL'], value='OTHER', inplace=True)
    unseentest_df['class'].replace(to_replace=['PNEUMO', 'NORMAL'], value='OTHER', inplace=True)

    train_generator = train_datagen.flow_from_dataframe(
        class_mode="binary",
        classes=['OTHER', 'COVID'],
        dataframe=train_df,
        directory="../input/covid19datasetcollectionsirm/",
        x_col="filename",
        y_col="class",
        target_size=(height, width),
        shuffle=True,
        seed=42,
        batch_size=16,
        color_mode='rgb'
    )

    validation_generator = validation_datagen.flow_from_dataframe(
        class_mode="binary",
        classes=['OTHER', 'COVID'],
        dataframe=dev_df,
        directory="../input/covid19datasetcollectionsirm/",
        x_col="filename",
        y_col="class",
        target_size=(height, width),
        shuffle=True,
        seed=42,
        batch_size=16,
        color_mode='rgb'
    )

    test_generator = test_datagen.flow_from_dataframe(
        class_mode="binary",
        classes=['OTHER', 'COVID'],
        dataframe=test_df,
        directory="../input/covid19datasetcollectionsirm/",
        x_col="filename",
        y_col="class",
        shuffle=False,
        target_size=(height, width),
        seed=42,
        batch_size=1,
        color_mode='rgb'
    )

    unseentest_generator = train_datagen.flow_from_dataframe(
        class_mode="binary",
        classes=['OTHER', 'COVID'],
        dataframe=unseentest_df,
        directory="../input/covid19datasetcollectionsirm/",
        x_col="filename",
        y_col="class",
        shuffle=False,
        target_size=(height, width),
        seed=42,
        batch_size=1,
        color_mode='rgb'
    )

    print(train_generator.n)

    #basemodel = DenseNet169(weights='imagenet', include_top=False, input_tensor=Input(shape=(width, height, 3)))
    basemodel = VGG16(input_tensor=Input(shape=(width, height, 3)),include_top=False, weights="imagenet")
    """
    mixed0 40
    mixed1 63
    mixed2 86
    mixed3 100
    mixed4 132
    mixed5 164
    mixed6 196
    mixed7 228
    mixed8 248
    mixed9_0 276
    mixed9 279
    mixed9_1 307
    mixed10 310
    """

    NB_IV3_LAYERS_TO_FREEZE = 63 
    for layer in basemodel.layers[:NB_IV3_LAYERS_TO_FREEZE]:
        layer.trainable = False
    for layer in basemodel.layers[NB_IV3_LAYERS_TO_FREEZE:]:
        layer.trainable = True

    headModel = basemodel.output
    headModel = MaxPooling2D(pool_size=(2, 2))(headModel)
    headModel = Flatten()(headModel)
    headModel = Dense(512, activation="relu")(headModel)
    headModel = BatchNormalization()(headModel)
    headModel = Dropout(0.25)(headModel)
    headModel = Dense(256, activation="relu")(headModel)
    headModel = BatchNormalization()(headModel)
    headModel = Dropout(0.25)(headModel)
    headModel = Dense(128, activation="relu")(headModel)
    headModel = BatchNormalization()(headModel)
    headModel = Dropout(0.25)(headModel)
    headModel = Dense(64, activation="relu")(headModel)
    headModel = BatchNormalization()(headModel)
    headModel = Dropout(0.25)(headModel)
    headModel = Dense(1, activation='sigmoid')(headModel)

    model = Model(inputs=basemodel.input, outputs=headModel)

    model.compile(loss="binary_crossentropy", optimizer=Adam(learning_rate = 1e-4, decay = learning_rate/epochs), metrics=["accuracy"])

    filepath = "best_InceptionV3_covid19_frezz_mmixed5_Config6_Holdout_Binary.h5"
    checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
    callbacks_list = [checkpoint]

    model.fit(
        train_generator,
        steps_per_epoch=train_generator.n // BS,
        epochs=epochs,
        validation_data=validation_generator,
        callbacks=callbacks_list,
        verbose=1
    )

    # Evaluating trained model on test set
    new_model = load_model(filepath)

    # Test set
    scores_t = new_model.evaluate(test_generator, verbose=0)
    predictions = new_model.predict(test_generator, use_multiprocessing=True)
    print(f"Accuracy using score: {scores_t[1] * 100}")

    cm_labels = ['OTHER', 'COVID']

    y_pred = list(predictions > 0.5)
    y_true = list(test_generator.classes)

    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    Sensitivity_t  = sensitivity_score(y_true, y_pred) * 100
    Specificity_t = specificity_score(y_true, y_pred) * 100

    print(classification_report(y_true, y_pred, target_names=cm_labels, digits=4))
    print(confusion_matrix(y_true, y_pred))
    print(f"Sensitivity : {Sensitivity_t}")
    print(f"Specificity : {Specificity_t}")

    # Unseen test set
    scores_u = new_model.evaluate(unseentest_generator, verbose=0)
    predictions = new_model.predict(unseentest_generator, use_multiprocessing=True)
    print(f"Accuracy using score: {scores_u[1] * 100}")

    cm_labels = ['OTHER', 'COVID']

    y_pred = list(predictions > 0.5)
    y_true = list(unseentest_generator.classes)

    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    Sensitivity_u = sensitivity_score(y_true, y_pred) * 100
    Specificity_u = specificity_score(y_true, y_pred) * 100

    print(classification_report(y_true, y_pred, target_names=cm_labels, digits=4))
    print(confusion_matrix(y_true, y_pred))
    print(f"Sensitivity : {Sensitivity_u}")
    print(f"Specificity : {Specificity_u}")

    # Validation set
    scores_v = new_model.evaluate(validation_generator, verbose=0)
    predictions = new_model.predict(validation_generator, use_multiprocessing=True)
    print(f"Accuracy using score: {scores_v[1] * 100}")

    cm_labels = ['OTHER', 'COVID']

    y_pred = list(predictions > 0.5)
    y_true = list(validation_generator.classes)

    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    Sensitivity_v  = sensitivity_score(y_true, y_pred) * 100
    Specificity_v = specificity_score(y_true, y_pred) * 100

    print(classification_report(y_true, y_pred, target_names=cm_labels, digits=4))
    print(confusion_matrix(y_true, y_pred))
    print(f"Sensitivity : {Sensitivity_v}")
    print(f"Specificity : {Specificity_v}")
   
    # Log metrics to MLflow
    mlflow.log_metric("Test Accuracy_Score", scores_t[1] * 100)
    mlflow.log_metric("Test Sensitivity_Score", Sensitivity_t)
    mlflow.log_metric("Test Specificity_Score", Specificity_t)
   
    mlflow.log_metric("Unseen Test Accuracy_Score", scores_u[1] * 100)
    mlflow.log_metric("Unseen Sensitivity_Score", Sensitivity_u)
    mlflow.log_metric("Unseen Specificity_Score", Specificity_u)
   
    mlflow.log_metric("Validation Accuracy_Score", scores_v[1] * 100)
    mlflow.log_metric("Validation Sensitivity_Score", Sensitivity_v)
    mlflow.log_metric("Validation Specificity_Score", Specificity_v)

    # Save the model artifact
    mlflow.keras.log_model(model, "model")

# End MLflow run
mlflow.end_run()

!pip install pyngrok
from pyngrok import ngrok
# Terminate open tunnels if exist
ngrok.kill()

# Setting the authtoken (optional)
# Get your authtoken from https://dashboard.ngrok.com/auth
NGROK_AUTH_TOKEN = "2YwjzxbGl7uRWqlQl7jDhS2RD8D_3CU9mF12mrRkh2pzHMUhk"
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

# Open an HTTPs tunnel on port 5000 for http://localhost:5000
ngrok_tunnel = ngrok.connect(addr="5000", proto="http", bind_tls=True)
print("MLflow Tracking UI:", ngrok_tunnel.public_url)

!mlflow ui

Found 2401 validated image filenames belonging to 2 classes.
Found 343 validated image filenames belonging to 2 classes.
Found 685 validated image filenames belonging to 2 classes.
Found 758 validated image filenames belonging to 2 classes.
2401
Epoch 1/4
150/150 [==============================] - 20s 122ms/step - loss: 0.5379 - accuracy: 0.7455 - val_loss: 0.3416 - val_accuracy: 0.9096

Epoch 00001: val_accuracy improved from -inf to 0.90962, saving model to best_InceptionV3_covid19_frezz_mmixed5_Config6_Holdout_Binary.h5
Epoch 2/4
150/150 [==============================] - 18s 122ms/step - loss: 0.3424 - accuracy: 0.8520 - val_loss: 0.1793 - val_accuracy: 0.9359

Epoch 00002: val_accuracy improved from 0.90962 to 0.93586, saving model to best_InceptionV3_covid19_frezz_mmixed5_Config6_Holdout_Binary.h5
Epoch 3/4
150/150 [==============================] - 18s 119ms/step - loss: 0.2500 - accuracy: 0.8981 - val_loss: 0.1576 - val_accuracy: 0.9388

Epoch 00003: val_accuracy improved from 

2024/02/21 09:55:46 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmp22s22h_u/model, flavor: keras), fall back to return ['tensorflow==2.6.2', 'keras==2.6.0']. Set logging level to DEBUG to see the full traceback.


MLflow Tracking UI: https://8d62-35-197-83-85.ngrok-free.app
[2024-02-21 09:55:59 +0000] [518] [INFO] Starting gunicorn 20.1.0
[2024-02-21 09:55:59 +0000] [518] [INFO] Listening at: http://127.0.0.1:5000 (518)
[2024-02-21 09:55:59 +0000] [518] [INFO] Using worker: sync
[2024-02-21 09:55:59 +0000] [521] [INFO] Booting worker with pid: 521
